In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
data=pd.read_csv("./merged/len3_ship_mon.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["Approved"])
data

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,Approved,KEGG,Ingredient,variable


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [3]:
data["elasped"]=data["year"]-data["Approved"]
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
# count ver.
# aggregated_data = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()

# dummy ver.
aggregated_df = pd.concat([
    data.groupby(['year', 'id']).first(), 
    dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int))
], axis=1).reset_index()
aggregated_df

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,Approved,KEGG,Ingredient,variable,elasped


In [4]:
colnames

[]

connect generic usage

In [5]:
dusage=pd.read_csv("./generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [6]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,Approved,KEGG,Ingredient,variable,elasped,Year,Percentage


In [7]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,Approved,KEGG,Ingredient,variable,elasped,Year,generic_per


In [8]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
df_with_dummies.to_csv("./merged/len3_ship_mon_agg_drop_never.csv",encoding="shift-jis")